In [41]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

from dash.exceptions import PreventUpdate

from collections import OrderedDict

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, Input, State, Output, callback

from jupyter_dash import JupyterDash

import numpy as np
import pandas as pd

import dash_bootstrap_components as dbc
COMPONENT_STYLE = "/assets/my_component.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [42]:
dt_changes = []

COLUMNS = ["apple", "pear", "orange"]

N_ROWS = 10

In [43]:
def create_data(columns, n_rows):

    size = n_rows * len(COLUMNS)

    data = np.random.randint(0, n_rows, size=size).reshape(n_rows, len(COLUMNS))

    df = pd.DataFrame(data=data, columns=COLUMNS)

    df.index.name = "row_id"

    return df.reset_index().to_dict(orient="records")

In [44]:
data_original = create_data(COLUMNS, N_ROWS)

In [45]:
data_original = [{'row_id': 0, 'apple': 5, 'pear': 5, 'orange': 2},
 {'row_id': 1, 'apple': 9, 'pear': 1, 'orange': 5},
 {'row_id': 2, 'apple': 7, 'pear': 1, 'orange': 2},
 {'row_id': 3, 'apple': 0, 'pear': 9, 'orange': 8},
 {'row_id': 4, 'apple': 9, 'pear': 3, 'orange': 2},
 {'row_id': 5, 'apple': 6, 'pear': 9, 'orange': 9},
 {'row_id': 6, 'apple': 5, 'pear': 5, 'orange': 4},
 {'row_id': 7, 'apple': 8, 'pear': 9, 'orange': 1},
 {'row_id': 8, 'apple': 0, 'pear': 0, 'orange': 3},
 {'row_id': 9, 'apple': 0, 'pear': 6, 'orange': 8}]

In [46]:
data_new = [{'row_id': 0, 'apple': 5, 'pear': 5, 'orange': 2},
 {'row_id': 1, 'apple': 9, 'pear': 1, 'orange': 5},
 {'row_id': 2, 'apple': 8, 'pear': 1, 'orange': 2},
 {'row_id': 3, 'apple': 0, 'pear': 10, 'orange': 8},
 {'row_id': 4, 'apple': 9, 'pear': 3, 'orange': 2},
 {'row_id': 5, 'apple': 6, 'pear': 9, 'orange': 9},
 {'row_id': 6, 'apple': 5, 'pear': 5, 'orange': 4},
 {'row_id': 7, 'apple': 8, 'pear': 9, 'orange': 1},
 {'row_id': 8, 'apple': 0, 'pear': 0, 'orange': 3},
 {'row_id': 9, 'apple': 0, 'pear': 6, 'orange': 8}]

In [47]:
df, df_previous = pd.DataFrame(data=data_new), pd.DataFrame(data=data_original)

In [48]:
df

,row_id,apple,pear,orange
0,0,5,5,2
1,1,9,1,5
2,2,8,1,2
3,3,0,10,8
4,4,9,3,2
5,5,6,9,9
6,6,5,5,4
7,7,8,9,1
8,8,0,0,3
9,9,0,6,8


In [49]:
df2 = [df, df_previous]
df2[1]

,row_id,apple,pear,orange
0,0,5,5,2
1,1,9,1,5
2,2,7,1,2
3,3,0,9,8
4,4,9,3,2
5,5,6,9,9
6,6,5,5,4
7,7,8,9,1
8,8,0,0,3
9,9,0,6,8


In [50]:
row_id_name="row_id"

In [51]:
for _df in [df, df_previous]:
    print('-------')
    print(_df)

-------
   row_id  apple  pear  orange
0       0      5     5       2
1       1      9     1       5
2       2      8     1       2
3       3      0    10       8
4       4      9     3       2
5       5      6     9       9
6       6      5     5       4
7       7      8     9       1
8       8      0     0       3
9       9      0     6       8
-------
   row_id  apple  pear  orange
0       0      5     5       2
1       1      9     1       5
2       2      7     1       2
3       3      0     9       8
4       4      9     3       2
5       5      6     9       9
6       6      5     5       4
7       7      8     9       1
8       8      0     0       3
9       9      0     6       8


In [52]:
for _df in [df, df_previous]:

    assert row_id_name in _df.columns

    _df = _df.set_index(row_id_name)

In [53]:
df

,row_id,apple,pear,orange
0,0,5,5,2
1,1,9,1,5
2,2,8,1,2
3,3,0,10,8
4,4,9,3,2
5,5,6,9,9
6,6,5,5,4
7,7,8,9,1
8,8,0,0,3
9,9,0,6,8


In [54]:
mask = df.ne(df_previous)
mask

,row_id,apple,pear,orange
0,False,False,False,False
1,False,False,False,False
2,False,True,False,False
3,False,False,True,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [55]:
df_diff = df[mask].dropna(how="all", axis="columns").dropna(how="all", axis="rows")
df_diff

,apple,pear
2,8.0,NaN
3,NaN,10.0


In [56]:
changes = []

for idx, row in df_diff.iterrows():

    row_id = row.name

    row.dropna(inplace=True)

    for change in row.iteritems():

        changes.append(
            {
                row_id_name: row_id,
                "column_name": change[0],
                "current_value": change[1],
                "previous_value": df_previous.at[row_id, change[0]],
            }
        )

In [57]:
changes

[{'row_id': 2,
  'column_name': 'apple',
  'current_value': 8.0,
  'previous_value': 7},
 {'row_id': 3,
  'column_name': 'pear',
  'current_value': 10.0,
  'previous_value': 9}]

In [58]:
def diff_dashtable(data, data_previous, row_id_name="row_id"):

    """Generate a diff of Dash DataTable data.

    Parameters
    ----------
    data: DataTable property (https://dash.plot.ly/datatable/reference)
        The contents of the table (list of dicts)
    data_previous: DataTable property
        The previous state of `data` (list of dicts).

    Returns
    -------
    A list of dictionaries in form of [{row_id_name:, column_name:, current_value:,
        previous_value:}]
    """

    df, df_previous = pd.DataFrame(data=data), pd.DataFrame(data_previous)

    for _df in [df, df_previous]:

        assert row_id_name in _df.columns

        _df = _df.set_index(row_id_name)

    mask = df.ne(df_previous)

    df_diff = df[mask].dropna(how="all", axis="columns").dropna(how="all", axis="rows")

    changes = []

    for idx, row in df_diff.iterrows():

        row_id = row.name

        row.dropna(inplace=True)

        for change in row.iteritems():

            changes.append(
                {
                    row_id_name: row_id,
                    "column_name": change[0],
                    "current_value": change[1],
                    "previous_value": df_previous.at[row_id, change[0]],
                }
            )

    return changes


In [59]:
app.layout = html.Div(
    [
        dcc.Store(id="diff-store"),
        html.P("Changes to DataTable:"),
        html.Div(id="data-diff"),
        html.Button("Apply Changes", id="button"),
        dash_table.DataTable(
            id="table-data-diff",
            columns=[{"id": col, "name": col, "type": "numeric"} for col in COLUMNS],
            editable=True,
            data=create_data(COLUMNS, N_ROWS),
            page_size=20,
        ),
    ]
)


@app.callback(
    Output("diff-store", "data"),
    [Input("table-data-diff", "data_timestamp")],
    [
        State("table-data-diff", "data"),
        State("table-data-diff", "data_previous"),
        State("diff-store", "data"),
    ],
)
def capture_diffs(ts, data, data_previous, diff_store_data):

    if ts is None:

        raise PreventUpdate

    diff_store_data = diff_store_data or {}

    diff_store_data[ts] = diff_dashtable(data, data_previous)

    return diff_store_data


@app.callback(
    Output("data-diff", "children"),
    [Input("button", "n_clicks")],
    [State("diff-store", "data")],
)
def update_output(n_clicks, diff_store_data):

    if n_clicks is None:

        raise PreventUpdate

    if diff_store_data:

        dt_changes = []

        for v in diff_store_data.values():

            dt_changes.append(f"* {v}")

        return [dcc.Markdown(change) for change in dt_changes]

    else:

        return "No Changes to DataTable"

In [60]:
app.run_server(host="0.0.0.0", port=5051, debug=True, use_reloader=False) 

/home/andrewcgaitskell/.local/share/virtualenvs/dmtoolnotes-z4lOQ0W-/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://0.0.0.0:5051/dash1/
